# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import os

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [ ]:
input_keyword = "網路爬蟲"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = str(input_keyword.encode('UTF-8')).upper()   # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [ ]:
# 模擬封包的標頭
headers = {
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [ ]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [72]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path':  key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    
        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        if not os.path.exists("./WikiArticle"):
            os.mkdir("./WikiArticle")
        with open(f'./WikiArticle/{key_word.replace('/',"-")}.html','w+',encoding='utf-8') as f:
            f.write(resp.text)

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        for p in  content:
            a_list = p.find_all('a',href=re.compile('^(/wiki/)(\S)*$'))
            for a in a_list:
                external_link_dict[a.get_text()] = a['href']

                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(key_word= k,key_word_link= v, recursive=recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

SyntaxError: f-string: mismatched '(', '{', or '[' (<ipython-input-72-491302f71741>, line 36)

### 執行前個步驟定義好的爬蟲主程式

In [ ]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[2] - /wiki/ViolaWWW (ViolaWWW)
遞迴層[2] - /wiki/Mosaic (Mosaic)
遞迴層[2] - /wiki/%E4%BC%8A%E5%88%A9%E8%AB%BE%E5%A4%A7%E5%AD%B8%E5%8E%84%E5%B7%B4%E7%B4%8D-%E9%A6%99%E6%AA%B3%E5%88%86%E6%A0%A1 (伊利諾大學厄巴納-香檳分校)
遞迴層[2] - /wiki/%E5%9B%BD%E5%AE%B6%E8%B6%85%E7%BA%A7%E7%94%B5%E8%84%91%E5%BA%94%E7%94%A8%E4%B8%AD%E5%BF%83 (國家超級電腦應用中心)
遞迴層[2] - /wiki/%E9%A9%AC%E5%85%8B%C2%B7%E5%AE%89%E5%BE%B7%E6%A3%AE (馬克·安德森)
遞迴層[2] - /wiki/%E8%89%BE%E7%88%BE%C2%B7%E9%AB%98%E7%88%BE (艾爾·高爾)
遞迴層[2] - /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91%E8%81%94%E7%9B%9F (全球資訊網協會)
遞迴層[2] - /wiki/%E9%BA%BB%E7%9C%81%E7%90%86%E5%B7%A5%E5%AD%A6%E9%99%A2 (麻省理工學院)
遞迴層[2] - /wiki/%E6%A8%99%E8%A8%98%E5%BC%8F%E8%AA%9E%E8%A8%80 (標記式語言)
遞迴層[2] - /wiki/%E8%AA%9E%E7%BE%A9%E7%B6%B2 (語義網)
遞迴層[2] - /wiki/%E5%90%8C%E4%B9%89%E8%AF%8D (同義詞)
遞迴層[2] - /wiki/TCP/IP (TCP/IP)


FileNotFoundError: [Errno 2] No such file or directory: './WikiArticle/TCP/IP.html'